In [1]:
import os
os.chdir('../../scripts/')
import somtools
import TransitSOM_release
import selfsom
from collections import Counter

In [2]:
K = TransitSOM_release.LoadSOM('../S6_outfile.txt',5,5,241)

In [3]:
array = K._K

In [4]:
## map the data onto a pixel
mapped_tuples = [tuple(point) for point in array]
## count how many are in that pixel
counts = Counter(mapped_tuples)
count_list = [counts[item] for item in mapped_tuples]

TypeError: unhashable type: 'numpy.ndarray'

In [6]:
mapped_tuples

[(array([-6.87018641e-05, -6.78869262e-05,  2.00950887e-05, -7.99640520e-05,
         -9.65154286e-05, -7.98240327e-05, -5.67298271e-05, -5.14579258e-05,
         -1.01969435e-04,  3.93106397e-05,  8.52063867e-05,  7.19137852e-05,
         -5.25010705e-05, -6.65414322e-05, -4.35752790e-05,  1.42408058e-04,
          3.75551718e-05,  1.13495069e-04, -1.11205214e-04, -1.77748115e-04,
         -1.27663528e-04, -1.64676419e-04, -8.60413131e-05, -1.38469771e-04,
         -1.79721395e-05, -8.26918540e-05,  1.59057973e-05,  1.31294252e-04,
          2.74191045e-04,  6.18316789e-05, -1.25032486e-05, -1.96736209e-05,
         -3.35185684e-05,  3.76261518e-05,  1.11666247e-04,  8.67764696e-05,
          1.01555239e-05, -8.78238828e-05, -1.16669674e-04,  1.36371216e-04,
          8.33891853e-05, -8.79045447e-06,  1.44797972e-04,  8.48596029e-05,
          2.36144750e-05, -1.25538325e-05, -3.77895634e-05,  8.69312651e-05,
          1.38826748e-04, -5.89498499e-05, -6.64854047e-05,  1.13549344e-05,